# Objetivo
Validar algumas transformações fora do spark.

In [1]:
import pandas as pd

In [2]:
config_types_mapping_filename = "/config/types_mapping.json"
data_filename_import = "/data/input/users/load.csv"
data_filename_parquet_product_export = "/data/output/product_export_without_spark.parquet"
data_filename_csv_product_export = "/data/output/product_export_without_spark.csv"

In [3]:
column_sort_values = "update_date"
column_drop_duplicates = "id"

In [4]:
df_row_data = pd.read_csv(data_filename_import)

In [5]:
# vizualização inicial dos dados brutos
df_row_data.head()

,id,name,email,phone,address,age,create_date,update_date
0,1,david.lynch@cognitivo.ai,David Lynch,(11) 99999-9997,"Mulholland Drive, Los Angeles, CA, US",72,2018-03-03 18:47:01.954752,2018-03-03 18:47:01.954752
1,1,david.lynch@cognitivo.ai,David Lynch,(11) 99999-9998,"Mulholland Drive, Los Angeles, CA, US",72,2018-03-03 18:47:01.954752,2018-04-14 17:09:48.558151
2,2,sherlock.holmes@cognitivo.ai,Sherlock Holmes,(11) 94815-1623,"221B Baker Street, London, UK",34,2018-04-21 20:21:24.364752,2018-04-21 20:21:24.364752
3,3,spongebob.squarepants@cognitivo.ai,Spongebob Squarepants,(11) 91234-5678,"124 Conch Street, Bikini Bottom, Pacific Ocean",13,2018-05-19 04:07:06.854752,2018-05-19 04:07:06.854752
4,1,david.lynch@cognitivo.ai,David Lynch,(11) 99999-9999,"Mulholland Drive, Los Angeles, CA, US",72,2018-03-03 18:47:01.954752,2018-05-23 10:13:59.594752


In [6]:
# verificando os dados do id 1.
df_row_data[df_row_data["id"] == 1].sort_values(column_sort_values)

,id,name,email,phone,address,age,create_date,update_date
0,1,david.lynch@cognitivo.ai,David Lynch,(11) 99999-9997,"Mulholland Drive, Los Angeles, CA, US",72,2018-03-03 18:47:01.954752,2018-03-03 18:47:01.954752
1,1,david.lynch@cognitivo.ai,David Lynch,(11) 99999-9998,"Mulholland Drive, Los Angeles, CA, US",72,2018-03-03 18:47:01.954752,2018-04-14 17:09:48.558151
4,1,david.lynch@cognitivo.ai,David Lynch,(11) 99999-9999,"Mulholland Drive, Los Angeles, CA, US",72,2018-03-03 18:47:01.954752,2018-05-23 10:13:59.594752


In [7]:
# ordenando dados
df_sorted_data = df_row_data.sort_values(column_sort_values)

# deduplicando dados
df_deduplicate_data = df_sorted_data.sort_values(column_sort_values).drop_duplicates(column_drop_duplicates,keep='last')

In [8]:
df_deduplicate_data.head()

,id,name,email,phone,address,age,create_date,update_date
2,2,sherlock.holmes@cognitivo.ai,Sherlock Holmes,(11) 94815-1623,"221B Baker Street, London, UK",34,2018-04-21 20:21:24.364752,2018-04-21 20:21:24.364752
5,3,spongebob.squarepants@cognitivo.ai,Spongebob Squarepants,(11) 98765-4321,"122 Conch Street, Bikini Bottom, Pacific Ocean",13,2018-05-19 04:07:06.854752,2018-05-19 05:08:07.964752
4,1,david.lynch@cognitivo.ai,David Lynch,(11) 99999-9999,"Mulholland Drive, Los Angeles, CA, US",72,2018-03-03 18:47:01.954752,2018-05-23 10:13:59.594752


In [9]:
# carregamento metodo para mapeamento de tipos
from load_mapping_types import load_mapping_types

types_mapping_dict = load_mapping_types(config_types_mapping_filename)
print(types_mapping_dict)

{'age': 'int', 'create_date': 'datetime64[ns]', 'update_date': 'datetime64[ns]'}


In [10]:
df_converted_type_data = df_deduplicate_data.astype(types_mapping_dict)
df_converted_type_data.head()

,id,name,email,phone,address,age,create_date,update_date
2,2,sherlock.holmes@cognitivo.ai,Sherlock Holmes,(11) 94815-1623,"221B Baker Street, London, UK",34,2018-04-21 20:21:24.364752,2018-04-21 20:21:24.364752
5,3,spongebob.squarepants@cognitivo.ai,Spongebob Squarepants,(11) 98765-4321,"122 Conch Street, Bikini Bottom, Pacific Ocean",13,2018-05-19 04:07:06.854752,2018-05-19 05:08:07.964752
4,1,david.lynch@cognitivo.ai,David Lynch,(11) 99999-9999,"Mulholland Drive, Los Angeles, CA, US",72,2018-03-03 18:47:01.954752,2018-05-23 10:13:59.594752


In [11]:
df_converted_type_data.dtypes

id                      int64
name                   object
email                  object
phone                  object
address                object
age                     int64
create_date    datetime64[ns]
update_date    datetime64[ns]
dtype: object

In [12]:
# convertendo datetime to timetamp devido o datetime não poder ser escrito diretamente em timestamp
from convert_pandas_datetime_to_timestamp import convert_pandas_datetime_to_timestamp

df_converted_type_data_to_timestamp = convert_pandas_datetime_to_timestamp(df_converted_type_data)

In [13]:
df_converted_type_data_to_timestamp.to_parquet(data_filename_parquet_product_export,index=False)
df_converted_type_data_to_timestamp.to_csv(data_filename_csv_product_export,index=False)

In [14]:
pd.read_parquet(data_filename_parquet_product_export).dtypes

id              int64
name           object
email          object
phone          object
address        object
age             int64
create_date     int64
update_date     int64
dtype: object